<a href="https://colab.research.google.com/github/chricannon/Predicting_popular_platforms/blob/main/Successful_Image_Processing_proj2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
from skimage.filters import prewitt_h, prewitt_v
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the directory paths
stroke_path = '/content/drive/MyDrive/Brain_Data_Organised/Stroke'
normal_path = '/content/drive/MyDrive/Brain_Data_Organised/Normal'

# Initialize the image list
image_list = []

# Function to list image files in a directory
def list_image_files(directory, label):
    image_files = []
    if os.path.exists(directory):
        for filename in os.listdir(directory):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                image_files.append((filename, label))
    else:
        print(f"Directory does not exist: {directory}")
    return image_files

# List files in the Normal directory and add to image_list
image_list.extend(list_image_files(normal_path, 'Normal'))

# List files in the Stroke directory and add to image_list
image_list.extend(list_image_files(stroke_path, 'Stroke'))

# Check if image_list is populated
if not image_list:
    print("No images found in the specified directories.")
else:
    print(f"Total images found: {len(image_list)}")
    print("Sample images:")
    for image_name, label in image_list[:5]:  # Print the first 5 images as a sample
        print(f"{label}: {image_name}")

# Define your parameters for LBP
n_points = 24
radius = 3
METHOD = 'uniform'

# Initialize a list to hold all the features
all_features = []

# Counters for summary
processed_images = 0
failed_images = 0

# Loop through each image, extract features, and store them
for image_name, label in image_list:
    image_path = os.path.join(normal_path if label == 'Normal' else stroke_path, image_name)
    try:
        print(f"Processing image: {image_path}")

        # Check if the image exists
        if not os.path.exists(image_path):
            print(f"Error: Image {image_path} does not exist.")
            failed_images += 1
            continue

        print(f"Reading image: {image_path}")
        image = imread(image_path)  # Read the image
        print(f"Image shape after reading: {image.shape}")

        print(f"Resizing image: {image_path}")
        image = resize(image, (100, 100))  # Resize image to 100x100 pixels
        print(f"Image shape after resizing: {image.shape}")

        # Handle grayscale and color images
        if len(image.shape) == 2:  # Grayscale image
            print(f"Image {image_path} is grayscale")
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB by repeating the channel
        else:
            print(f"Image {image_path} is RGB or has multiple channels")

        # Extract RGB features by averaging the pixel values
        rgb_feature_matrix = np.mean(image, axis=2)
        rgb_features = rgb_feature_matrix.flatten()  # Flatten to 1D array

        # Convert image to grayscale for further feature extraction
        gray_image = rgb2gray(image)

        # Extract Local Binary Pattern features
        lbp_image = local_binary_pattern(gray_image, n_points, radius, METHOD)
        lbp_features = lbp_image.flatten()

        # Extract horizontal and vertical edge features using Prewitt filters
        horizontal_edges = prewitt_h(gray_image).flatten()
        vertical_edges = prewitt_v(gray_image).flatten()

        # Combine all features into a single array
        features = np.concatenate([rgb_features, lbp_features, horizontal_edges, vertical_edges])
        all_features.append(features)

        # Print the features to verify
        if processed_images < 5:  # Print only the first 5 for brevity
            print(f"Features for image {image_path}: {features[:10]}...")  # Print the first 10 elements for brevity

        processed_images += 1

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        failed_images += 1

# Convert the list of features to a NumPy array
X = np.array(all_features)

# Print the shape of X to verify
print(f"\nShape of feature matrix X: {X.shape}")

# Create the labels array
y = np.array([label for _, label in image_list])

# Print the shape of y to verify
print(f"Shape of label array y: {y.shape}")

# Print a few labels to verify
print("Sample labels from y:", y[:10])

# Summary of processing
print(f"\nTotal images processed: {processed_images}")
print(f"Total images failed: {failed_images}")

Mounted at /content/drive
Total images found: 2501
Sample images:
Normal: 108 (20).jpg
Normal: 126 (4).jpg
Normal: 114 (6).jpg
Normal: 116 (23).jpg
Normal: 116 (11).jpg
Processing image: /content/drive/MyDrive/Brain_Data_Organised/Normal/108 (20).jpg
Reading image: /content/drive/MyDrive/Brain_Data_Organised/Normal/108 (20).jpg
Image shape after reading: (650, 650)
Resizing image: /content/drive/MyDrive/Brain_Data_Organised/Normal/108 (20).jpg
Image shape after resizing: (100, 100)
Image /content/drive/MyDrive/Brain_Data_Organised/Normal/108 (20).jpg is grayscale
Features for image /content/drive/MyDrive/Brain_Data_Organised/Normal/108 (20).jpg: [7.27737465e-06 1.24364107e-04 1.68688124e-04 1.68934082e-04
 1.68934082e-04 1.68934082e-04 1.68934082e-04 1.68934082e-04
 1.68934082e-04 1.68934082e-04]...
Processing image: /content/drive/MyDrive/Brain_Data_Organised/Normal/126 (4).jpg
Reading image: /content/drive/MyDrive/Brain_Data_Organised/Normal/126 (4).jpg


/usr/local/lib/python3.10/dist-packages/skimage/feature/texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Processing image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/72 (13).jpg
Reading image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/72 (13).jpg
Image shape after reading: (650, 650)
Resizing image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/72 (13).jpg
Image shape after resizing: (100, 100)
Image /content/drive/MyDrive/Brain_Data_Organised/Stroke/72 (13).jpg is grayscale
Processing image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/81 (4).jpg
Reading image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/81 (4).jpg
Image shape after reading: (650, 650)
Resizing image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/81 (4).jpg
Image shape after resizing: (100, 100)
Image /content/drive/MyDrive/Brain_Data_Organised/Stroke/81 (4).jpg is grayscale
Processing image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/71 (16).jpg
Reading image: /content/drive/MyDrive/Brain_Data_Organised/Stroke/71 (16).jpg
